<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex13notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex13notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo13.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

----
## 演習問題: 外食支出金額データのクラスタリング / $K$-平均法の実験
----

次の二つのことをやってみましょう：

1. 「SSDSE-家計消費」データを階層型クラスタリング手法でクラスタリングする
1. 人工的に作った2次元データを $K$-平均法でクラスタリングする

<b><font color="#ff0000">
注意:
今回の notebook の中には，コードセルを実行すると問題の解答が表示されるようになっている箇所があります．
</font>
</b>

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の階層型クラスタリングパッケージ
import scipy.cluster.hierarchy as hierarchy

# scikit-learn の K-平均法のクラス
from sklearn.cluster import KMeans

# 「解答」を示す際に文字列を復号するのに使う
import base64
# 復号した文字列を Markdown 形式で（数式は LaTeX でフォーマットして）表示
from IPython.display import display, Markdown

---
### 外食支出金額データのクラスタリング

主成分分析の回でも使った「SSDSE-家計消費」データに階層型クラスタリング手法を適用してみましょう．

#### データを準備する

In [ ]:
# データを扱うためのプログラムを入手
!wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/DiningOutSpendingData.py
# それをインポートして実行
from DiningOutSpendingData import DiningOutSpendingData
dosd = DiningOutSpendingData(dropcolumns=True)
dosd.info()
# DataFrame を入手して表示
dfDining = dosd.getDataFrame()
dfDining

次のコードセルでは，読み込んだデータを NumPy array にして，変数ごとに標準化する前処理を行っています．

In [ ]:
# NumPy array を作る
Xtmp = dosd.getArray()

# 標準化する
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_Dining = ss.fit_transform(Xtmp)
N_Dining, D_Dining = X_Dining.shape

# 最初の5つを表示
print(X_Dining[:5, :])
print(X_Dining.shape)

---
#### 階層型クラスタリングしてみよう



上記の配列 `X_Dining` を学習データとして，階層型クラスタリングを実行します．
[scipy.cluster.hiearchy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) でクラスタリングして，
[scipy.cluster.hierarchy.dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) でデンドログラムを描きます．
標本間の距離はユークリッド距離，クラスタ間の距離はウォード法で測ります．

In [ ]:
# 階層型クラスタリングを実行
link = hierarchy.linkage(X_Dining, method='ward', metric='euclidean')
# デンドログラムを描く
fig, ax = plt.subplots(facecolor="white", figsize=(12, 6))
hierarchy.dendrogram(link, distance_sort='descending', show_leaf_counts=True, leaf_font_size=16, ax=ax)
plt.show()

以下のセルを実行すると，`threshold` で指定した値をクラスタ間距離のしきい値として，データをクラスタに分けた結果を表示します．表示される値のうち，`label` の列がクラスタの番号に相当します（0ではなく1からはじまる番号）．

In [ ]:
threshold = 12 # クラスタ間距離のしきい値

label = hierarchy.fcluster(link, t=threshold, criterion='distance')
df_result = dosd.city.iloc[1:, :].copy()
df_result['label'] = label
df_result['idx'] = np.arange(47, dtype=int)
df_result = df_result.set_index('idx')
df_result = df_result.sort_values('label')
df_result[['label','Prefecture', 'City']]

#### 問題

次の問に対する答えを考えよう．

(1) この実験では何をやっている？どんな手法を使っている？

(2) 使っているデータのデータ数と次元数はいくつ？


In [ ]:
# このセルを実行すると，上記の問に対する解答例が表示されます
Q = b'CigxKSDlpJblh7rmlK/lh7rph5HpoY3jg4fjg7zjgr/jga7jgq/jg6njgrnjgr/jg6rjg7PjgrDjgpLooYzjgaPjgabjgYTjgovvvI7pmo7lsaTlnovjgq/jg6njgrnjgr/jg6rjg7PjgrDvvIjjga7kuK3jgafjgoLlh53pm4blnovjgq/jg6njgrnjgr/jg6rjg7PjgrDvvInjga7miYvms5XjgpLnlKjjgYTjgabjgYrjgorvvIzmqJnmnKzplpPjga7ot53pm6Ljga/jg6bjg7zjgq/jg6rjg4Pjg4not53pm6LjgafvvIzjgq/jg6njgrnjgr/plpPjga7ot53pm6Ljga/jgqbjgqnjg7zjg4nms5XjgaflrprjgoHjgabjgYTjgovvvI4KCigyKSDjg4fjg7zjgr/mlbDjga8gNDcg44Gn5qyh5YWD5pWw44GvIDnvvI5gWERpbmluZy5zaGFwZWAg44Gu5YCk44KSIHByaW50IOOBl+OBpuOBhOOCi+OCs+ODvOODieOCu+ODq+OBruWun+ihjOe1kOaenOOCkuimi+OCi+OBqOWIhuOBi+OCi++8jgo='
display(Markdown(base64.b64decode(Q).decode('utf-8')))

(3) クラスタ間距離のしきい値を 12 とした場合，このデータはいくつのクラスタに分けられるか．また，しきい値が 6 のときはいくつに分けられるか．

(4) 名古屋市と外食支出の傾向がよく似ている都市を2つ挙げなさい．




In [ ]:
# このセルを実行すると，上記の問に対する解答例が表示されます
Q = b'CigzKSDkuIroqJjjga7jg4fjg7Pjg4njg63jgrDjg6njg6Djgafjga/jgq/jg6njgrnjgr/plpPot53pm6Ljga7jgZfjgY3jgYTlgKTjgYznuKbou7jjgavooajjgZXjgozjgabjgYTjgovvvI7nuKbou7jjga7lgKQgMTIg44Gn5qiq6Lu444Go5bmz6KGM44Gq55u057ea44KS5byV44GP44Go77yMM+acrOOBruaeneOBqOS6pOOCj+OCi++8jgrjgZfjgZ/jgYzjgaPjgabvvIzjgq/jg6njgrnjgr/plpPot53pm6Ljga7jgZfjgY3jgYTlgKTjgpIgMTIg44Go44GZ44KL44Go77yM44OH44O844K/44GvIDMg44Gk44Gu44Kv44Op44K544K/44Gr5YiG44GR44KJ44KM44KL77yOCuWQjOanmOOBq++8jOOBl+OBjeOBhOWApOOBjCA2IOOBruOBqOOBjeOBr++8jDnlgIvjga7jgq/jg6njgrnjgr/jgavliIbjgZHjgonjgozjgovvvI4K5a6f6Zqb44GrIGB0aHJlc2hvbGRgIOOBruWApOOCkiAxMiDjgYrjgojjgbMgNiDjgajjgZfjgabjgq/jg6njgrnjgr/jg6rjg7PjgrDntZDmnpzjgpLooajnpLrjgZnjgovjgrPjg7zjg4njgrvjg6vjgpLlrp/ooYzjgZfvvIzjgZ3jga7pgJrjgorjga7ntZDmnpzjgYzlvpfjgonjgozjgovjgZPjgajjgpLnorroqo3jgZfjgojjgYbvvI4KCig0KSDlkI3lj6TlsYvluILjga8gMjIg55Wq77yO44OH44Oz44OJ44Ot44Kw44Op44Og44KS6KaL44KL44Go77yM44Kv44Op44K544K/6ZaT6Led6Zui44Gu44GX44GN44GE5YCk44KS44GC44GS44Gm44GE44Gj44Gf44Go44GN77yMMjIg55Wq44Gv5pyA5Yid44GrIDIwIOeVqu+8iOWykOmYnOecjOWykOmYnOW4gu+8ieOBqOWQiOS9teOBl++8jOasoeOBqyAxMiDnlarvvIjmnbHkuqzpg73ljLrpg6jvvInjgajlkIjkvbXjgZfjgabjgYTjgovvvI4K44Gd44Gu5b6M44Gv44GX44GN44GE5YCk44KS44GL44Gq44KK5aSn44GN44GP44GZ44KL44G+44Gn44GT44GuM+mDveW4guOBruOBv+OBp+S4gOOBpOOBruOCr+ODqeOCueOCv+OCkuaIkOOBl+OBpuOBhOOCi++8jgrjgZfjgZ/jgYzjgaPjgabvvIzlkI3lj6TlsYvluILjga7lpJbpo5/mlK/lh7rlgr7lkJHjga/vvIzlspDpmJznnIzlspDpmJzluILjgYrjgojjgbPmnbHkuqzpg73ljLrpg6jjgajjgojjgY/kvLzjgabjgYTjgovjgajjgYTjgYjjgovvvI4K'
display(Markdown(base64.b64decode(Q).decode('utf-8')))

(5) デンドログラムを見て，クラスタの数がそれぞれ 4, 6, 9 となるようなしきい値を選ぼう．それぞれのしきい値設定でクラスタリングした結果を観察しよう．

(6) (5) の結果から考えると，近畿5府県（滋賀，京都，大阪，兵庫，奈良）の県庁所在市の外食支出傾向は，互いに似てる？ 似てない？ それぞれのクラスタ数の条件で同じクラスタに入っているかどうか見てみよう．

(7) 47県庁所在市の中で，他の市との違いが大きくて最も孤立しているのはどこだろう？
孤立した点でなくなる（他のクラスタと合併される）ときのクラスタ間距離が最も大きい市を探そう．

In [ ]:
# このセルを実行すると，上記の問に対する解答例が表示されます
Q = b'Cig1KSDjg4fjg7Pjg4njg63jgrDjg6njg6DjgavmqKrou7jjgajlubPooYzjgarnm7Tnt5rjgpLlvJXjgYTjgabjgb/jgovjgajvvIzkvovjgYjjgbDvvIznuKbou7jjga7lgKTjgYwgMTAsIDgsIDYg44Gu44Go44GN44Gr77yM44Gd44KM44Ge44KMIDQsIDYsIDkg5pys44Gu5p6d44Go5Lqk44KP44KL77yOCuOBl+OBn+OBjOOBo+OBpu+8jOOCr+ODqeOCueOCv+mWk+i3nembouOBruOBl+OBjeOBhOWApOOCkuS+i+OBiOOBsCAxMCwgOCwgNiDjgajjgZnjgozjgbDjgojjgYTvvIjjgIzoprPlr5/jgZfjgojjgYbjgI3jgavlr77jgZnjgovop6PnrZTkvovjga/nnIHnlaXvvInvvI4KCig2KSDov5Hnlb815bqc55yM44Gv77yM44Kv44Op44K544K/6ZaT6Led6Zui44Gu44GX44GN44GE5YCk44KSIDYg56iL5bqm44G+44Gn5LiL44GS44Gm44Kv44Op44K544K/5pWw44GMIDkg44Go44Gq44Gj44Gm44KC5ZCM44GY44Kv44Op44K544K/44Gr5bGe44GX44Gm44GE44KLCu+8iOWun+mam+OBq+OBr+OCguOBhuWwkeOBl+OBl+OBjeOBhOWApOOCkuS4i+OBkuOBpuOCguWQjOOBmOOCr+ODqeOCueOCv+OBq+WxnuOBl+OBpuOBhOOCi++8ie+8juOBl+OBn+OBjOOBo+OBpu+8jOWklumjn+aUr+WHuuWCvuWQkeOBr+S8vOOBpuOBhOOCi+OBqOWIpOaWreOBp+OBjeOCi++8jgoKKDcpIOOCr+ODqeOCueOCv+mWk+i3nembouOBruOBl+OBjeOBhOWApOOCkuWkp+OBjeOBj+OBl+OBpuOBhOOBo+OBn+OBqOOBje+8jOacgOW+jOOBq++8iOacgOOCguWkp+OBjeOBhOOBl+OBjeOBhOWApOOBp++8ieWtpOeri+OBl+OBn+eCueOBp+OBquOBj+OBquOCi+OBruOBryAzNiDnlarvvIjpppnlt53nnIzpq5jmnb7luILvvInjgafjgYLjgovvvI4K'
display(Markdown(base64.b64decode(Q).decode('utf-8')))

---
### $K$-平均法の実験

非階層型クラスタリングの代表的手法である$K$-平均法について，簡単な実験を通してちょびっとだけ学びましょう．$K$-平均法では，データをいくつのクラスタに分けるかをあらかじめ指定する必要があります．また，クラスタリングの過程で，各クラスタに所属するデータの平均（これをセントロイドといいます）を計算します．クラスタ数を $K$ とすると，$K$ 個の平均(mean)を求めることになるので，$K$-means 法と呼ばれます．

#### データを準備する

In [ ]:
# 実験用データの入手
df = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/data4kmeans.csv')
X = df[['x1', 'x2']].to_numpy()
print('最初の5件')
print(X[:5, :], X.shape)

# 散布図
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(X[:, 0], X[:, 1])
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_aspect('equal')
plt.show()

2次元ベクトル100個から成るデータをクラスタリングしてみましょう．散布図は上の通りです．なんとなく3つくらいの塊があるように見えますが...．

#### K-means クラスタリングを実行して結果を観察しよう

ここでは，scikit-learn の [sklearn.cluster.KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) クラスを使い，上記の2次元データを $K$-平均法でクラスタリングする実験をやってみましょう．



次のセルの `K` にクラスタ数を指定して実行すると，配列 `X` に格納されたデータのクラスタリングに対して $K$-平均法を適用した結果を表示します．

In [ ]:
K = 2 #@param  {type: "number"}

# K-means 法
km = KMeans(n_clusters=K, verbose=0)
km.fit(X) # 学習
centroid = km.cluster_centers_ # セントロイド
label = km.predict(X) # 各データの所属クラスタを求める
print(f'K = {K}')
for ik in range(K):
    print(f'cluster{ik}: num = {np.sum(label == ik)}')

# クラスタリング結果描画用のデータ
xmin, xmax = -5, 5
ymin, ymax = -5, 5
p = np.dstack(np.mgrid[xmin:xmax:0.05, ymin:ymax:0.05])
P = p.reshape((-1, p.shape[2]))
labelP = km.predict(P)

# クラスタ割り振り結果を描画
colors = seaborn.color_palette(n_colors=K)
fig = plt.figure(figsize=(10, 6))
ax0 = fig.add_subplot(121)
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')
for ik in range(K):
    Xk = X[label==ik, :]
    Pk = P[labelP==ik, :]
    ax0.scatter(Xk[:, 0], Xk[:, 1], color=colors[ik], s=12)
    ax0.plot(centroid[ik, 0], centroid[ik, 1], color='white', marker='*', markerfacecolor=colors[ik], markersize=25)
    ax0.scatter(Pk[:, 0], Pk[:, 1], marker='.', alpha=0.05, color=colors[ik])

plt.show()

`clusterX: num = N` という表示は，データ点のうち，`X` 番目のクラスタに属すと判断されたものが `N` 個あったことを意味します．`K` 個のクラスタにほぼ同数ずつ割り振られることもあれば，偏ることもあります．

図は，各データ点および平面上の各点をクラスタに振り分けた結果を色分けして示しています．また，各クラスタのセントロイドを ★ で表示しています．セントロイドは，そのクラスタに所属すると判断されたデータたちの平均（重心）です．

`K` の値をいろいろ変えて結果を観察しましょう．ちなみに，$K$-平均法のアルゴリズムにはランダムな処理が含まれますので，同じデータで同じ`K`の設定でも，実行のたびに結果が変わります．